In [2]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [ ]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
api_key = "67565f211247a70ca588f9bb4e9561aa.WKluNGHOe5UR4Ot8"
llm_model = "glm-4"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_base="https://open.bigmodel.cn/api/paas/v4/",
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../ragtest/output/20240917-211927/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 8
Report count after filtering by community level 2: 8


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,0,# HUST Information Security Class and Yuan Bin...,0,3.5,HUST Information Security Class and Yuan Bin's...,The impact severity rating is moderate due to ...,This community is centered around the relation...,[{'explanation': 'Yuan Bin is an associate pro...,"{\n ""title"": ""HUST Information Security Cla...",d50c169d-d9b3-4957-a695-b769091fc6a8
1,1,# Zhou Pan and Online Machine Learning Communi...,0,3.5,Zhou Pan and Online Machine Learning Community,The impact severity rating is moderate due to ...,"This community is centered around Zhou Pan, a ...",[{'explanation': 'Zhou Pan is a central figure...,"{\n ""title"": ""Zhou Pan and Online Machine L...",33421b98-e4ea-42fd-8b5a-62550f7b134a
2,2,# Wen Ming and Program Analysis & Security Com...,0,3.5,Wen Ming and Program Analysis & Security Commu...,The impact severity rating is moderate due to ...,"This community is centered around Wen Ming, an...",[{'explanation': 'Wen Ming is a key entity in ...,"{\n ""title"": ""Wen Ming and Program Analysis...",ab3d70b5-4c34-4c59-bcd4-faea02ccecf7
3,3,# HUST Information Security Community\n\nThis ...,0,4.5,HUST Information Security Community,The impact severity rating is moderate-to-high...,"This community is centered around Mei Song, a ...",[{'explanation': 'Mei Song is a key figure in ...,"{\n ""title"": ""HUST Information Security Com...",e853749f-4cb8-4bf9-9037-ecbe5ccb8cf1
4,4,# Wang Haoyu and Mobile Application Security C...,0,4.5,Wang Haoyu and Mobile Application Security Com...,The impact severity rating is moderate due to ...,"This community is centered around Wang Haoyu, ...",[{'explanation': 'Wang Haoyu is a key entity i...,"{\n ""title"": ""Wang Haoyu and Mobile Applica...",a55eb659-5fb9-4261-af1b-9afad7b39f6e


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
result = await search_engine.asearch(
    "介绍一下网络空间安全课程"
)

print(result.response)

网络空间安全课程在我国的华中科技大学有着深厚的实践与教学基础。以下是对该课程的综合介绍：

### 课程社区

课程主要围绕几个关键社区展开，其中包括：

- **网络空间安全综合实践（V）社区** [Data: Reports (5)]
- **信息安全社区** [Data: Reports (3)]
- **芯片安全社区** [Data: Reports (6)]

这些社区以教授和讲师为核心，涵盖了网络空间安全、信息安全、芯片安全等多个方面。

### 实践与教育

- **网络空间安全综合实践（V）课程** 强调实践方面的网络安全知识，对塑造学生的技能和网络安全知识可能发挥关键作用。
  
- **信息安全社区** 以梅松教授为中心，他教授的网络编程安全课程对学生的网络空间安全理解和实践具有重要影响。

- **芯片安全社区** 以王洪飞研究员为中心，专注于芯片安全的教育，对学生未来的职业发展和技术能力具有重要影响。

- 另有以肖玲副教授为中心的 **信息安全课程社区**，强调网络安全教育，对学生未来在网络安全领域的职业发展具有重要影响 [Data: Reports (7)]。

### 教育影响

这些课程和社区不仅提供了丰富的网络安全知识，还通过实践和指导，帮助学生深入理解网络空间安全的重要性，为未来的职业发展打下坚实基础。

综合来看，华中科技大学的网络空间安全课程具有以下特点：

- 实践性强，注重学生技能的培养。
- 由资深教授和讲师领导，确保教学质量。
- 涵盖多个子领域，为学生提供全面的网络安全教育。

这些课程和社区在培养我国网络空间安全人才方面发挥着重要作用。


In [10]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,5,HUST Cyberspace Security Comprehensive Practic...,1.0,# HUST Cyberspace Security Comprehensive Pract...,6.5
1,3,HUST Information Security Community,1.0,# HUST Information Security Community\n\nThis ...,4.5
2,6,Wang Hongfei and Chip Security Community,1.0,# Wang Hongfei and Chip Security Community\n\n...,4.5
3,7,HUST Information Security Course Community,1.0,# HUST Information Security Course Community\n...,4.5
4,4,Wang Haoyu and Mobile Application Security Com...,1.0,# Wang Haoyu and Mobile Application Security C...,4.5
5,1,Zhou Pan and Online Machine Learning Community,1.0,# Zhou Pan and Online Machine Learning Communi...,3.5
6,2,Wen Ming and Program Analysis & Security Commu...,1.0,# Wen Ming and Program Analysis & Security Com...,3.5
7,0,HUST Information Security Class and Yuan Bin's...,1.0,# HUST Information Security Class and Yuan Bin...,3.5


In [11]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 2. LLM tokens: 5577
